In [ ]:
project_folder = ''

In [5]:
from custompackage.load_data import *
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
# override SSL certificate verification failure
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [6]:
bs = 256
weighting = 'paired'
trials = 10
classes = 10
ds_set = ['mnist', 'fmnist', 'kmnist', 'svhn', 'usps', 'cifar10']

#paired_test = np.zeros((len(ds_set), trials, 10, 10))
paired_test = np.load(project_folder+"results/confused_pairs_all.npy", allow_pickle=True)

# For each 10-class dataset
for k, ds in enumerate(ds_set):
    # Go through each class
    for t1 in range(classes):

        # and pair it with every other class
        for t2 in range(t1 + 1, classes):

            if paired_test[k, -1, t1, t2] != 0:
                continue

            # Load the binary classification dataloaders
            trainloaders, validloaders, testloader = dataset_weighted_split_all(bs, t1, t2, weighting, trials, ds)
            X_test = testloader.dataset.tensors[0]
            y_test = testloader.dataset.tensors[1]

            for m in range(trials):
                # Assign entirety of the datasets within each dataloader to a variable
                X_train = trainloaders[m].dataset.tensors[0]
                y_train = trainloaders[m].dataset.tensors[1]

                # initialize lda
                lda = LinearDiscriminantAnalysis()

                # fit to images, labels
                lda.fit(X_train, y_train)

                # see accuracy for validation set
                score_test = lda.score(X_test, y_test)

                print(ds, m, t1, t2, score_test)

                #Record keeping

                paired_test[k, m, t1, t2] = score_test

                np.save(project_folder+'results/confused_pairs_all.npy', paired_test)

  0%|          | 0/6579383 [00:00<?, ?it/s]

  0%|          | 0/1831726 [00:00<?, ?it/s]

usps 0 0 1 0.9181380417335474
usps 1 0 1 0.8812199036918138
usps 2 0 1 0.8860353130016051
usps 3 0 1 0.898876404494382
usps 4 0 1 0.9309791332263242
usps 5 0 1 0.8956661316211878
usps 6 0 1 0.913322632423756
usps 7 0 1 0.8812199036918138
usps 8 0 1 0.9085072231139647
usps 9 0 1 0.9341894060995185
usps 0 0 2 0.9245960502692998
usps 1 0 2 0.895870736086176
usps 2 0 2 0.9030520646319569
usps 3 0 2 0.9210053859964094
usps 4 0 2 0.8994614003590664
usps 5 0 2 0.895870736086176
usps 6 0 2 0.8886894075403949
usps 7 0 2 0.9120287253141831
usps 8 0 2 0.9156193895870736
usps 9 0 2 0.9102333931777379
usps 0 0 3 0.9295238095238095
usps 1 0 3 0.9219047619047619
usps 2 0 3 0.9104761904761904
usps 3 0 3 0.9352380952380952
usps 4 0 3 0.8914285714285715
usps 5 0 3 0.9047619047619048
usps 6 0 3 0.9428571428571428
usps 7 0 3 0.9333333333333333
usps 8 0 3 0.9466666666666667
usps 9 0 3 0.9352380952380952
usps 0 0 4 0.9355992844364938
usps 1 0 4 0.9552772808586762
usps 2 0 4 0.9409660107334525
usps 3 0 4 0.9

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifardata\cifar-10-python.tar.gz to ./cifardata
Files already downloaded and verified
cifar10 0 0 1 0.673
cifar10 1 0 1 0.659
cifar10 2 0 1 0.6785
cifar10 3 0 1 0.676
cifar10 4 0 1 0.6835
cifar10 5 0 1 0.665
cifar10 6 0 1 0.6945
cifar10 7 0 1 0.685
cifar10 8 0 1 0.663
cifar10 9 0 1 0.701
Files already downloaded and verified
Files already downloaded and verified
cifar10 0 0 2 0.6475
cifar10 1 0 2 0.6235
cifar10 2 0 2 0.6395
cifar10 3 0 2 0.6305
cifar10 4 0 2 0.675
cifar10 5 0 2 0.627
cifar10 6 0 2 0.639
cifar10 7 0 2 0.6465
cifar10 8 0 2 0.641
cifar10 9 0 2 0.638
Files already downloaded and verified
Files already downloaded and verified
cifar10 0 0 3 0.695
cifar10 1 0 3 0.679
cifar10 2 0 3 0.706
cifar10 3 0 3 0.6735
cifar10 4 0 3 0.666
cifar10 5 0 3 0.6555
cifar10 6 0 3 0.684
cifar10 7 0 3 0.673
cifar10 8 0 3 0.673
cifar10 9 0 3 0.671
Files already downloaded and verified
Files already downloaded and verified
cifar10 0 0 4 0.6985
cifar10 1 0 4 0.691
cifar10 2 0 4 0.6885
c